In [1]:
import tweepy
from tweepy import Cursor
import pandas as pd
import numpy as np
from datetime import datetime, date, time, timedelta
from collections import Counter

### add your twitter keys
Replace the CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET in the code below with your own credentials

In [11]:
twitter_keys = {
        'consumer_key':        'CONSUMER_KEY',
        'consumer_secret':     'CONSUMER_SECRET',
        'access_token_key':    'ACCESS_TOKEN',
        'access_token_secret': 'ACCESS_TOKEN_SECRET'
    }

auth = tweepy.OAuthHandler(twitter_keys['consumer_key'], twitter_keys['consumer_secret'])
auth.set_access_token(twitter_keys['access_token_key'], twitter_keys['access_token_secret'])

api = tweepy.API(auth)

In [4]:
id_to_label = {
    80676829: 'Krzysztof Bosak',
    955239446: 'Władysław Kosiniak-Kamysz',
    2280346687: 'Małgorzata Kidawa-Błońska',
    1193947151971311618: 'Szymon Hołownia',
    202086424: 'Andrzej Duda',
    466781777: 'Robert Biedroń'}

candidates_ids = [key for key in id_to_label.keys()]

## Daily stats
Get todays number of followers, following and tweets for every candidate and save to csv

In [ ]:
daily_stats = 'daily-stats.csv'

In [5]:
def get_current_stats():
    today = datetime.today().strftime('%d/%m/%Y')
    candidates = [api.get_user(id = user_id) for user_id in candidates_ids]
    idx = pd.MultiIndex.from_product([[user.id for user in candidates], [today]], names=['profile', 'date'])
    col = ['followers', 'following', 'tweets']
    df = pd.DataFrame(
        [[user.followers_count, user.friends_count, user.statuses_count] for user in candidates],
            idx, col)
    return df


def update_stats_file():
    loaded_data = pd.read_csv(daily_stats, sep='\t').set_index(['profile', 'date'])
    today = datetime.today().strftime('%d/%m/%Y')
    if not today in loaded_data.index.levels[1]:
        todays_stats = get_current_stats()
        updated = pd.concat([loaded_data, todays_stats])
        updated.to_csv(daily_stats, sep="\t")
        print("Csv file updated for: " + today)
    print("Data is up to date")

In [6]:
# update_stats_file()

Data is up to date


## Tweets history

In [7]:
from datetime import date

DATE_FORMAT = "%d/%m/%Y"

def get_tweets_from_period(user_id, from_date, till_date, save=False, replace=False):
    tweets = []
    start_date = datetime.strptime(from_date, DATE_FORMAT).date()
    end_date = datetime.strptime(till_date, DATE_FORMAT).date()

    # get tweets from API
    for status in Cursor(api.user_timeline, id=user_id).items():
        if status.created_at.date() > end_date:
            continue
        if status.created_at.date() < start_date:
            break
        if status.truncated:    
            try :
                status = api.get_status(status.id_str, tweet_mode='extended')
            except tweepy.TweepError as error :
                if(error.__dict__['api_code'] == 34):
                    accountStatus = 'dead'
                    print("...{} is dead".format(status.id_str))
                    continue
                else:
                    raise
        else:
            status.full_text = status.text
        tweets.append(status)
        
    # tweets to data frame
    col = ['id','user_id','date','created_at',
           'is_retweet', 'is_quote', 'likes', 'retweets', 
           'hashtags', 'mentions','text'
          ]
    series_list = [pd.Series([
            tweet.id,
            tweet.user.id,
            tweet.created_at.strftime(DATE_FORMAT),
            str(tweet.created_at),
            int(hasattr(tweet, "retweeted_status")),
            int(tweet.is_quote_status),
            tweet.favorite_count,
            tweet.retweet_count,
            ', '.join(hashtag['text'] for hashtag in tweet.entities['hashtags']),
            ', '.join(user['screen_name'] for user in tweet.entities["user_mentions"]),
            tweet.full_text
        ], index=col) for tweet in tweets]

    df = pd.DataFrame(series_list)
    df = df.replace("", "-")
    df = df.set_index('id')
    if save:
        file_name = "{}-tweets.csv".format(user_id)
        if replace:
            df.to_csv(file_name, sep="\t")
        else:
            saved_df = pd.read_csv(file_name, sep='\t', index_col=0)
            updated_df = pd.concat([df, saved_df]).reset_index().drop_duplicates(subset=['id']).set_index('id')
            print("saved: {}, new: {}".format(len(saved_df), len(updated_df)))
            updated_df.to_csv(file_name, sep="\t")
    return df

In [10]:
for user_id in candidates_ids:
    get_tweets_from_period(user_id ,'10/04/2020', '14/04/2020', save=True, replace=True)